In [1]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output

preds_dir = '/home/ubuntu/data/iceberg/results/final_preds/mix'

In [2]:
sub_path = preds_dir
all_files = os.listdir(sub_path)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5
0,5941774d,0.002232,0.116469,0.100000,0.001687,0.429033,0.001788
1,4023181e,0.941467,0.973660,0.844906,0.952506,0.748257,0.948093
2,b20200e4,0.935035,0.000052,0.133093,0.778264,0.031417,0.003917
3,e7f018bb,1.000000,0.999718,0.900000,0.997760,0.985124,0.999534
4,4371c8c3,0.186414,0.926600,0.829293,0.273599,0.662400,0.979448


In [3]:
# check correlation
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5
is_iceberg_0,1.000000,0.741182,0.612090,0.688893,0.331162,0.551109
is_iceberg_1,0.741182,1.000000,0.740887,0.777035,0.479192,0.724837
is_iceberg_2,0.612090,0.740887,1.000000,0.778494,0.669184,0.849344
is_iceberg_3,0.688893,0.777035,0.778494,1.000000,0.619875,0.816598
is_iceberg_4,0.331162,0.479192,0.669184,0.619875,1.000000,0.761113
is_iceberg_5,0.551109,0.724837,0.849344,0.816598,0.761113,1.000000


In [4]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 0:5].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 0:5].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 0:5].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 0:5].median(axis=1)

In [5]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

In [2]:
# load the model with best base performance
sub_base = pd.read_csv(preds_dir + '/cnn_train_0.96_best.csv')

In [3]:
sub_base.shape

(8424, 2)

In [7]:
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_bestbase_2.csv', 
                                        index=False, float_format='%.6f')

In [8]:
import os
os.getcwd()

'/home/ubuntu/git/learningWithKaggle/ice'

In [9]:
preds = sub_base.is_iceberg

In [14]:
# sub_base

In [11]:
submit_nums = np.clip(preds, 0.1, 0.9)

In [16]:
submission = pd.DataFrame({'id': sub_base.id, 'is_iceberg': submit_nums.reshape((submit_nums.shape[0]))})
print(submission.head(10))

submission.to_csv('/home/ubuntu/data/iceberg/results/final_preds/cnn_096_best_clip.csv', index=False)

         id  is_iceberg
0  5941774d    0.100000
1  4023181e    0.900000
2  b20200e4    0.100000
3  e7f018bb    0.900000
4  4371c8c3    0.900000
5  a8d9b1fd    0.310728
6  29e7727e    0.100000
7  92a51ffb    0.900000
8  c769ac97    0.100000
9  aee0547d    0.100000


/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [17]:
sub_base = pd.read_csv('/home/ubuntu/git/learningWithKaggle/ice/cnn_with_haralicks_0.96.csv')

In [19]:
sub_base.is_iceberg.max(), sub_base.is_iceberg.min()

(0.99416309595108043, 0.00078504218254238378)

In [20]:
sub_base2 = pd.read_csv('/home/ubuntu/git/learningWithKaggle/ice/cnn_with_haralicks_clipped0.96.csv')

In [21]:
sub_base2.is_iceberg.max(), sub_base2.is_iceberg.min()

(0.89999997615814209, 0.10000000149011612)